# Cálculo de probabilidades

Sea $(\Omega, \mathcal{E}, \mathbb{P})$ un espacio de probabilidad clásico:
$$
\begin{align}
    \mathbb{P}&:\mathcal{E}\to [0,1]\\
        E& \mapsto \frac{\# E}{\# \Omega}
\end{align}
$$      

Recordemos que cada realización de un evento es equiprobable.

El módulo `fractions` provee soporte para aritmética de números racionales.

In [12]:
from fractions import Fraction

def P(evento, espacio): 
    "La probabilidad de un evento."
    return Fraction(len(evento & espacio), 
                    len(espacio))

In [13]:
# Probabilidad de obtener un número par en un lanzamiento de un dado
D    = {1, 2, 3, 4, 5, 6}
par = {   2,    4,    6}

P(par, D)

Fraction(1, 2)

## Urnas

Una urna contiene $23$ bolas: $8$ blancas (W), $6$ azúles (B), and $9$ rojas (R). Seleccionamos $6$ bolas al azar. Calculemos la probabilidad de los siguientes eventos:
1. todas sean rojas.
2. $3$ sean azules, $2$ sean blancas, and $1$ sea roja.
3. exactamente $4$ sean blancas.

Primero, etiquetamos cada bola en la urna:

In [2]:
def conc(A, B):

    return {a + b 
            for a in A for b in B}

urna = conc('W', '12345678') | conc('B', '123456') | conc('R', '123456789') 

urna

{'B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'R1',
 'R2',
 'R3',
 'R4',
 'R5',
 'R6',
 'R7',
 'R8',
 'R9',
 'W1',
 'W2',
 'W3',
 'W4',
 'W5',
 'W6',
 'W7',
 'W8'}

In [5]:
len(urna)

23

A continuación, vamos a definir nuestro **espacio de estados**. El método `itertools.combinations(iterable, r)`:
    
Retorna subsecuencias de longitud $r$ con elementos del iterable de entrada.

Las tuplas de combinación se emiten en orden lexicográfico según el orden de la entrada iterable. Entonces, si la entrada iterable está ordenada, las tuplas de combinación se producirán en una secuencia ordenada.

Los elementos son tratados como únicos basados en su posición, no en su valor. De esta manera, si los elementos de entrada son únicos, no habrá valores repetidos en cada combinación

In [4]:
import itertools

def comb(items, n):

    return {' '.join(combo) 
            for combo in itertools.combinations(items, n)}

U6 = comb(urna, 6)


### Solución a 1.

In [8]:
r6 = {s for s in U6 if s.count('R') == 6}

P(r6, U6)

Fraction(4, 4807)

### Solución a 2.

In [9]:
b3w2r1 = {s for s in U6 if
          s.count('B') == 3 and s.count('W') == 2 and s.count('R') == 1}

P(b3w2r1, U6)

Fraction(240, 4807)

### Solución a 3.

In [10]:
w4 = {s for s in U6 if
      s.count('W') == 4}

P(w4, U6)

Fraction(350, 4807)

## Lanzamiento de un dado

In [8]:
def even(n): 
    return n % 2 == 0

### Predicados

Es necesario modificar la función $\mathbb{P}$ para aceptar un conjunto de resultados, o un predicado sobre los resultados (una función que regrese un booleano TRUE para un resultado que sea un evento).

Por ejemplo, supongamos que lanzamos un dado. Queremos definir el evento: El resultado del número que se obtiene es par. En este caso, el conjunto asociado al evento es:
        $$\{n\in \{1,2,3,4,5,6\} \ \text{tal que} \ n\cong 0 \ \text{mod} \ 2  \}=\{2,4,6\}$$

In [5]:
def P(evento, espacio): 
    
    if es_predicado(evento):
        evento = tal_que(evento, espacio)
    return Fraction(len(evento & espacio), len(espacio))

es_predicado = callable

def tal_que(predicado, colección): 
    
    return {e for e in colección if predicado(e)}

In [6]:
es_predicado

<function callable(obj, /)>

In [14]:
tal_que(even, D)

{2, 4, 6}

## El problema de Newton

Se tienen las siguientes tres proposiciones:

1. Seis dados se lanzan y se obtiene al menos un $6$.
2. Doce dados se lanzan y se obtienen al menos dos $6$.
3. Dieciocho dados se lanzan y se obtienen al menos tres $6$.

¿Cuál de las proposiciones anteriores tiene mayor probabilidad de ocurrir?

Para resolver el problema, es necesario introducir un diccionario para trabajar con las leyes (o distribuciones) de las variables aleatorias que se conciben.

Recordemos que si $X:\Omega\to R_{X}$ es una variable aleatoria discreta definida sobre un espacio de probabilidad clásico, entonces su distribución de probabilidades es
    $$\sum_{k\in R_{X}}\mathbb{P}(X=k)=1.$$

In [15]:
class DistProb(dict):
    def __init__(self, map=(), **kwargs): # kwargs en una función se usa para pasar, de forma opcional, un número variable de argumentos con nombre.
        self.update(map, **kwargs)
        total = sum(self.values())
        for outcome in self:
            self[outcome] = self[outcome] / total
            assert self[outcome] >= 0

In [16]:
def P(evento, espacio): 
    if es_predicado(evento):
        evento = tal_que(evento, espacio)
    if isinstance(espacio, DistProb):
        return sum(espacio[o] for o in espacio if o in evento)
    else:
        return Fraction(len(evento & espacio), len(espacio))
    
def such_that(predicado, espacio): 
    if isinstance(espacio, DistProb):
        return DistProb({o:espacio[o] for o in espacio if predicado(o)})
    else:
        return {o for o in espacio if predicado(o)}

In [20]:
def joint(A, B, sep=''):
    return DistProb({a + sep + b: A[a] * B[b]
                    for a in A
                    for b in B})

In [21]:
dado = DistProb({'6':1/6, '-':5/6})

def dados(n, dado):
    if n == 1:
        return dado
    else:
        return joint(dado, dados(n - 1, dado))

In [22]:
dados(3, dado)

{'666': 0.0046296296296296285,
 '66-': 0.023148148148148143,
 '6-6': 0.023148148148148143,
 '6--': 0.11574074074074073,
 '-66': 0.023148148148148143,
 '-6-': 0.11574074074074073,
 '--6': 0.11574074074074073,
 '---': 0.5787037037037037}

In [25]:
def al_menos(k, resultado): 
    return lambda s: s.count(resultado) >= k # Las expresiones lambda en Python son una forma corta de declarar funciones pequeñas y anónimas

**Solución a 1**

In [26]:
P(al_menos(1, '6'), dados(6, dado))

0.6651020233196159

**Solución 2**

In [27]:
P(al_menos(2, '6'), dados(12, dado))

0.6186673737323101

**Solución 3**

In [28]:
P(al_menos(3, '6'), dados(18, dado))

0.5973456859477544